In [1]:
import os, sys
import pandas as pd
import numpy as np

path = os.getcwd()

In [2]:

ffname="czapla2022"


In [3]:
full_dataset = pd.read_csv("czapla2022_3sig_tet_mpn", index_col=0)
full_dataset = full_dataset[~full_dataset.dimer_neighbors.str.contains('\.')].reset_index(drop=True)
#full_dataset = full_dataset[['step_dimer','step_tetramer','tilt','roll','twist','shift','slide','rise']].rename(columns={"step_dimer":'step'})
print(len(full_dataset))

detdf   = pd.read_csv("czapla2022_pdbid_data", index_col=False)

ff_dataset = pd.read_csv(ffname+"_3sig_tet", index_col=0)
ff_dataset = ff_dataset[~ff_dataset.dimer_neighbors.str.contains('\.')].reset_index(drop=True)
#ff_dataset = ff_dataset[['step_dimer','step_tetramer','tilt','roll','twist','shift','slide','rise']].rename(columns={"step_dimer":'step'})
print(len(ff_dataset))

92716
53350


### lists and functions:

In [4]:
theta_lst = ['tilt','roll','twist','shift','slide','rise']
# --- Lists of dimers, tetramers, and the self-complementary steps ---
BASES     = ['A','C','G','T']
COMP      = {'A':'T', 'T':'A', 'C':'G', 'G':'C','.':'.'}
BASES     = ['A','C','G','T']
COMP      = {'A':'T', 'T':'A', 'C':'G', 'G':'C','.':'.'}

def dna_seq_complement(sequence_string):
    COMP = {'A':'T', 'T':'A', 'C':'G', 'G':'C','.':'.'}
    STEP = sequence_string[::-1]
    STEP = ''.join([COMP[STEP[i]] for i in range(len(STEP))])
    return STEP

DIMERS    = [b+c for b in BASES for c in BASES]
DIM_MATCH = {'TT':'AA', 'GT':'AC', 'CT':'AG', 'CC':'GG', 'TC':'GA', 'TG':'CA'}

lstRR = ['AG','GG','AA','GA']
lstYR = ['CG','CA','TA']
lstRY = ['AT','AC','GC']
DIMER_LST = ['CG','CA','TA'] + ['AG','GG','AA','GA'] + ['AT','AC','GC']

SCDIM     = []
for DIM in DIMERS:
    if COMP[DIM[1]]+COMP[DIM[0]]==DIM:
        SCDIM.append(DIM)

TET_LST      = [a+b+c+d for a in ['A','C','G','T','.'] for b in BASES for c in BASES for d in ['A','C','G','T','.']]
TETRAMERS    = [i for i in TET_LST if '.' not in i]
TETRAMER_LST = [
    'AAAA','AACA','AAGA','AATA','ACAA','ACGA','AGAA','AGCA','AGGA','ATAA',
    'AAAC','AACC','AAGC','AATC','ACAC','ACGC','AGAC','AGCC','AGGC','ATAC',
    'AAAG','AACG','AAGG','AATG','ACAG','ACGG','AGAG','AGCG','AGGG','ATAG',
    'AAAT','AACT','AAGT','AATT','ACAT','ACGT','AGAT','AGCT','AGGT','ATAT',
    'CAAA','CACA','CAGA','CATA','CCAA','CCGA','CGAA','CGCA','CGGA','CTAA',
    'CAAC','CACC','CAGC','CCAC','CGAC','CGGC','CAAG','CACG','CAGG','CATG',
    'CCAG','CCGG','CGAG','CGCG','CGGG','CTAG','CAAT','CACT','CAGT','CCAT',
    'CGAT','CGGT','GAAA','GACA','GAGA','GATA','GCAA','GCGA','GGAA','GGCA',
    'GGGA','GTAA','GAAC','GACC','GAGC','GATC','GCAC','GCGC','GGAC','GGCC',
    'GGGC','GTAC','GAAG','GACG','GAGG','GATG','GCAG','GCGG','GGAG','GGCG',
    'GGGG','GTAG','GAAT','GACT','GAGT','GCAT','GGAT','GGGT','TAAA','TACA',
    'TAGA','TATA','TCAA','TCGA','TGAA','TGCA','TGGA','TTAA','TAAC','TACC',
    'TAGC','TCAC','TGAC','TGGC','TAAG','TACG','TAGG','TCAG','TGAG','TGGG',
    'TAAT','TACT','TAGT','TCAT','TGAT','TGGT'
]
SCTET        = []
for TET in TETRAMERS:
    if COMP[TET[3]]+COMP[TET[2]]+COMP[TET[1]]+COMP[TET[0]]==TET and '.' not in TET:
        SCTET.append(TET)

### Analyze datasets

In [58]:
df_struc_avg = pd.DataFrame(index=full_dataset.step_dimer.unique())

for STEP in full_dataset.step_dimer.unique():
    df_struc_avg.at[STEP, "Count"] = len(full_dataset.loc[(full_dataset.step_dimer==STEP)
                                                      &(full_dataset.collection_selfcomp!='mod')])
    
    for THETA in theta_lst:
        df_struc_avg.at[STEP, THETA]              = round(full_dataset.loc[(full_dataset.step_dimer==STEP)][THETA].mean(), 3)
        df_struc_avg.at[STEP, "("+THETA+" std)"]  = round(full_dataset.loc[(full_dataset.step_dimer==STEP)][THETA].std(),  3)
        
df_struc_avg['Count'] = df_struc_avg['Count'].astype(int)

df_struc_avg.to_csv(ffname+"_dimer-step_par-details_struc-space")

df_struc_avg

,Count,tilt,(tilt std),roll,(roll std),twist,(twist std),shift,(shift std),slide,(slide std),rise,(rise std)
AA,2943,-0.017,3.099,-0.058,4.830,35.253,3.772,0.007,0.453,-0.257,0.398,3.243,0.180
TT,2732,0.017,3.099,-0.058,4.830,35.253,3.772,-0.007,0.453,-0.257,0.398,3.243,0.180
GT,2417,0.006,3.255,1.600,3.829,32.213,3.834,0.001,0.581,-0.598,0.415,3.260,0.209
AC,2712,-0.006,3.255,1.600,3.829,32.213,3.834,-0.001,0.581,-0.598,0.415,3.260,0.209
AG,2664,-0.319,3.742,3.187,4.314,32.342,4.089,-0.027,0.599,-0.311,0.507,3.307,0.225
CT,2358,0.319,3.742,3.187,4.314,32.342,4.089,0.027,0.599,-0.311,0.507,3.307,0.225
AT,3778,0.000,2.550,0.036,3.732,30.987,3.377,-0.000,0.460,-0.670,0.294,3.229,0.166
CA,2845,-0.026,3.788,5.485,4.923,34.993,4.372,-0.053,0.603,0.195,0.733,3.336,0.237
TG,2513,0.026,3.788,5.485,4.923,34.993,4.372,0.053,0.603,0.195,0.733,3.336,0.237
CG,3834,-0.000,4.848,6.435,5.059,34.001,4.209,0.000,0.600,0.371,0.625,3.341,0.270


In [59]:
uni_df_struc_avg = pd.DataFrame(index=full_dataset.DIMER.unique())

for STEP in full_dataset.DIMER.unique():
    uni_df_struc_avg.at[STEP, "Count"] = len(full_dataset.loc[(full_dataset.DIMER==STEP)])/2
    
    for THETA in theta_lst:
        uni_df_struc_avg.at[STEP, THETA]              = round(full_dataset.loc[(full_dataset.DIMER==STEP)][THETA].mean(), 3)
        uni_df_struc_avg.at[STEP, "("+THETA+" std)"]  = round(full_dataset.loc[(full_dataset.DIMER==STEP)][THETA].std(),  3)
        
uni_df_struc_avg['Count'] = uni_df_struc_avg['Count'].astype(int)

uni_df_struc_avg.to_csv(ffname+"_dimer-step-unique_par-details_struc-space")

uni_df_struc_avg

,Count,tilt,(tilt std),roll,(roll std),twist,(twist std),shift,(shift std),slide,(slide std),rise,(rise std)
AA,5675,-0.0,3.098,-0.058,4.830,35.253,3.772,-0.0,0.453,-0.257,0.398,3.243,0.180
AC,5129,-0.0,3.254,1.600,3.829,32.213,3.834,-0.0,0.581,-0.598,0.415,3.260,0.209
AG,5022,-0.0,3.756,3.187,4.314,32.342,4.089,0.0,0.600,-0.311,0.507,3.307,0.225
AT,3778,0.0,2.550,0.036,3.732,30.987,3.377,-0.0,0.460,-0.670,0.294,3.229,0.166
CA,5358,0.0,3.788,5.485,4.922,34.993,4.372,0.0,0.605,0.195,0.733,3.336,0.237
CG,3834,-0.0,4.848,6.435,5.059,34.001,4.209,0.0,0.600,0.371,0.625,3.341,0.270
GA,5353,0.0,3.593,1.937,4.082,36.307,4.015,0.0,0.545,-0.082,0.479,3.285,0.199
GC,3499,0.0,3.760,2.450,3.845,33.902,3.867,0.0,0.602,-0.385,0.442,3.296,0.222
GG,5103,0.0,4.043,4.721,4.353,33.293,4.088,-0.0,0.678,-0.279,0.637,3.360,0.229
TA,3607,-0.0,3.776,2.164,6.849,37.417,5.773,0.0,0.642,0.245,0.874,3.344,0.230


In [60]:
i = 0
for STEP in uni_df_struc_avg.index:
    if STEP not in SCDIM:
        i+= 2*uni_df_struc_avg.at[STEP, 'Count']
    else:
        i+= uni_df_struc_avg.at[STEP, 'Count']
print(i)
del i

77998


In [61]:
df_seq_avg = pd.DataFrame(index=full_dataset.step_dimer.unique())

for STEP in full_dataset.step_dimer.unique():
    
    df = full_dataset.copy().loc[full_dataset.step_dimer==STEP].reset_index(drop=True)
    
    if STEP in SCDIM:
        df_seq_avg.at[STEP, 'Count']=len(df)/2
    else:
        df_seq_avg.at[STEP, 'Count']=len(df)
    tet_lst = df.step_tetramer.unique().tolist() 
    
    for THETA in theta_lst:
        
        avgs_array = np.zeros( len(tet_lst) )
        
        for i in range(len(tet_lst)):
            avgs_array[i] = df.loc[df.step_tetramer==tet_lst[i]][THETA].mean()
        
        df_seq_avg.at[STEP, THETA] = round( np.average(avgs_array), 3)
        del avgs_array
    del df, tet_lst
        
df_seq_avg['Count'] = df_seq_avg['Count'].astype(int)
df_seq_avg.to_csv(ffname+"_dimer-step_par-details_seq-space")

df_seq_avg

,Count,tilt,roll,twist,shift,slide,rise
AA,5675,0.010,0.180,35.060,0.009,-0.245,3.237
TT,5675,-0.010,0.180,35.060,-0.009,-0.245,3.237
GT,5129,0.040,1.467,32.291,0.003,-0.602,3.264
AC,5129,-0.040,1.467,32.291,-0.003,-0.602,3.264
AG,5022,-0.264,3.114,32.424,-0.030,-0.309,3.313
CT,5022,0.264,3.114,32.424,0.030,-0.309,3.313
AT,3778,-0.000,0.147,31.031,0.000,-0.669,3.229
CA,5358,-0.054,5.320,34.979,-0.046,0.236,3.331
TG,5358,0.054,5.320,34.979,0.046,0.236,3.331
CG,3834,0.000,6.518,33.910,0.000,0.381,3.329


In [62]:
uni_df_seq_avg = pd.DataFrame(index=full_dataset.DIMER.unique())

for STEP in full_dataset.DIMER.unique():
    
    df = full_dataset.copy().loc[full_dataset.DIMER==STEP].reset_index(drop=True)
    
    if STEP in SCDIM:
        uni_df_seq_avg.at[STEP, 'Count']=len(df)/2
    else:
        uni_df_seq_avg.at[STEP, 'Count']=len(df)
        
    tet_lst = df.step_tetramer.unique().tolist() 
    
    for THETA in theta_lst:
        
        avgs_array = np.zeros( len(tet_lst) )
        
        for i in range(len(tet_lst)):
            avgs_array[i] = df.loc[df.step_tetramer==tet_lst[i]][THETA].mean()
        
        uni_df_seq_avg.at[STEP, THETA] = round( np.average(avgs_array), 3)
        del avgs_array
    del df, tet_lst
        
uni_df_seq_avg['Count'] = uni_df_seq_avg['Count'].astype(int)
uni_df_seq_avg.to_csv(ffname+"_dimer-step-unique_par-details_seq-space")

uni_df_seq_avg

,Count,tilt,roll,twist,shift,slide,rise
AA,11350,-0.0,0.180,35.060,0.0,-0.245,3.237
AC,10258,0.0,1.467,32.291,-0.0,-0.602,3.264
AG,10044,-0.0,3.114,32.424,0.0,-0.309,3.313
AT,3778,-0.0,0.147,31.031,0.0,-0.669,3.229
CA,10716,0.0,5.320,34.979,-0.0,0.236,3.331
CG,3834,0.0,6.518,33.910,0.0,0.381,3.329
GA,10706,-0.0,1.872,36.326,0.0,-0.058,3.294
GC,3499,0.0,2.538,33.809,0.0,-0.382,3.283
GG,10206,-0.0,4.675,33.260,0.0,-0.297,3.361
TA,3607,-0.0,2.180,37.409,-0.0,0.227,3.345


In [63]:
for theta in theta_lst:
    print(theta, 
          str(round(df_struc_avg[theta].mean(), 3)), 
          str(round(uni_df_struc_avg[theta].mean(), 3)), 
          str(round(df_seq_avg[theta].mean(), 3)), 
          str(round(uni_df_seq_avg[theta].mean(), 3)))

tilt 0.0 0.0 0.0 0.0
roll 2.802 2.796 2.79 2.801
twist 34.069 34.071 34.052 34.05
shift 0.0 0.0 0.0 0.0
slide -0.194 -0.177 -0.187 -0.172
rise 3.299 3.3 3.299 3.299


### MpN averages for unculled dataset

In [48]:
dataset = pd.read_csv("czapla-xray-2022_dataset", index_col=0)
dataset = dataset[~dataset.dimer_neighbors.str.contains('\.')].reset_index(drop=True)
#full_dataset = full_dataset[['step_dimer','step_tetramer','tilt','roll','twist','shift','slide','rise']].rename(columns={"step_dimer":'step'})
print(len(dataset))

55713


In [52]:
mpnprime_dataset = pd.DataFrame(index=DIMER_LST)
for DIMER in DIMERS:
    for THETA1 in theta_lst:
        mpnprime_dataset.at[DIMER, THETA1] = round(dataset.loc[(dataset.step_dimer==DIMER)][THETA1].mean(), 3)
mpnprime_dataset

,tilt,roll,twist,shift,slide,rise
CG,0.000,6.377,33.982,-0.000,0.338,3.349
CA,-0.295,5.469,35.263,-0.193,0.180,3.340
TA,-0.000,2.096,37.204,0.000,0.265,3.340
AG,-1.967,3.388,32.303,0.086,-0.306,3.308
GG,-0.720,4.570,33.402,-0.142,-0.274,3.367
AA,-1.306,-0.128,35.166,-0.013,-0.247,3.244
GA,-1.584,1.926,36.131,-0.292,-0.074,3.274
AT,-0.000,0.005,31.172,-0.000,-0.665,3.234
AC,0.028,1.457,32.313,0.209,-0.618,3.263
GC,-0.000,2.474,33.839,0.000,-0.360,3.286


In [53]:
for theta in theta_lst:
    print(mpnprime_dataset[theta].sum(), mpnprime_dataset[theta].sum()/16)


-11.075000000000001 -0.6921875000000001
44.493 2.7808125
545.1030000000001 34.068937500000004
-0.483 -0.0301875
-2.9459999999999997 -0.18412499999999998
52.80500000000001 3.3003125000000004


### MpN data from pdb_time-series-data

In [7]:
datapath = path+'/pdb_time-series-data'

In [29]:
yeardf   = pd.DataFrame(columns=[2000+2*i for i in range(0, 12)], 
                        index=['count']+[i for i in theta_lst])

for YEAR in [2000+2*i for i in range(0, 12)]:
    
    lst = detdf.loc[detdf.deposit_year<=YEAR].pdb_id.unique().tolist()
    
    dataset = full_dataset[full_dataset.pdb_id.isin(lst)].reset_index(drop=True)
    
    mpn_dataset = pd.DataFrame(index=DIMERS)
    ct = 0
    for DIMER in DIMERS:
        ct += len(dataset.loc[(dataset.step_dimer==DIMER)&(dataset.collection_selfcomp!='mod')])
        for THETA1 in theta_lst:
            mpn_dataset.at[DIMER, THETA1] = round(dataset.loc[(dataset.step_dimer==DIMER)][THETA1].mean(), 3)    
    
    yeardf.at['count',YEAR]=ct
    del ct
    
    for theta in theta_lst:
        yeardf.at[theta,YEAR] = round(mpn_dataset[theta].sum()/16, 3)
    #del dataset, mpn_dataset

yeardf.to_csv(datapath+"/yearly_mpn_data")
#del yeardf

In [30]:
mpn_dataset

,tilt,roll,twist,shift,slide,rise
AA,-0.017,-0.058,35.253,0.007,-0.257,3.243
AC,-0.006,1.600,32.213,-0.001,-0.598,3.260
AG,-0.319,3.187,32.342,-0.027,-0.311,3.307
AT,0.000,0.036,30.987,-0.000,-0.670,3.229
CA,-0.026,5.485,34.993,-0.053,0.195,3.336
CC,0.077,4.721,33.293,0.007,-0.279,3.360
CG,-0.000,6.435,34.001,0.000,0.371,3.341
CT,0.319,3.187,32.342,0.027,-0.311,3.307
GA,-0.070,1.937,36.307,-0.029,-0.082,3.285
GC,0.000,2.450,33.902,0.000,-0.385,3.296


In [31]:
yeardf

,2000,2002,2004,2006,2008,2010,2012,2014,2016,2018,2020,2022
count,2498,3811,5435,8118,10085,13735,18427,23481,29757,37620,45253,46358
tilt,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
roll,3.488,3.293,3.123,2.938,2.969,2.939,3.026,2.99,2.922,2.833,2.796,2.802
twist,33.845,33.965,34.047,34.048,33.999,34.012,33.962,33.973,33.987,34.032,34.078,34.069
shift,-0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0
slide,-0.185,-0.187,-0.198,-0.208,-0.21,-0.196,-0.2,-0.21,-0.212,-0.2,-0.191,-0.194
rise,3.319,3.314,3.31,3.309,3.308,3.303,3.301,3.302,3.299,3.298,3.3,3.3
